## 예측 모델 수행하기

### 3.1 데이터 탐색

- 종속변수인 자전거 대여량이 **연속형 변수**이므로 **예측 모델**을 선택하여 데이터 학습 진행

#### 데이터 가져오기

In [6]:
import pandas as pd

X_train = pd.read_csv('bigData-main/bike_x_train.csv', encoding='cp949')
X_test  = pd.read_csv('bigData-main/bike_x_test.csv', encoding='cp949')
y_train = pd.read_csv('bigData-main/bike_y_train.csv', encoding='cp949')

In [7]:
# 독립변수 : X_train, X_test
print(X_train.head(5))
print(X_test.head(5))

          datetime  계절  공휴일  근무일  날씨    온도    체감온도  습도   풍속
0  2011-01-01 0:00   1    0    0   1  9.84  14.395  81  0.0
1  2011-01-01 1:00   1    0    0   1  9.02  13.635  80  0.0
2  2011-01-01 2:00   1    0    0   1  9.02  13.635  80  0.0
3  2011-01-01 3:00   1    0    0   1  9.84  14.395  75  0.0
4  2011-01-01 4:00   1    0    0   1  9.84  14.395  75  0.0
          datetime  계절  공휴일  근무일  날씨     온도    체감온도  습도       풍속
0  2011-01-20 0:00   1    0    1   1  10.66  11.365  56  26.0027
1  2011-01-20 1:00   1    0    1   1  10.66  13.635  56   0.0000
2  2011-01-20 2:00   1    0    1   1  10.66  13.635  56   0.0000
3  2011-01-20 3:00   1    0    1   1  10.66  12.880  56  11.0014
4  2011-01-20 4:00   1    0    1   1  10.66  12.880  56  11.0014


In [8]:
# 종속변수 : y_train
print(y_train.head())

          datetime  count
0  2011-01-01 0:00     16
1  2011-01-01 1:00     40
2  2011-01-01 2:00     32
3  2011-01-01 3:00     13
4  2011-01-01 4:00      1


#### 행/열 확인하기

In [9]:
print(X_train.shape, X_test.shape, y_train.shape)

(10886, 9) (6493, 9) (10886, 2)


#### 요약정보 확인하기

- 결측치 없음
- object : datetime

In [10]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  10886 non-null  object 
 1   계절        10886 non-null  int64  
 2   공휴일       10886 non-null  int64  
 3   근무일       10886 non-null  int64  
 4   날씨        10886 non-null  int64  
 5   온도        10886 non-null  float64
 6   체감온도      10886 non-null  float64
 7   습도        10886 non-null  int64  
 8   풍속        10886 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 765.5+ KB
None


In [13]:
# 범주형 변수들 값 확인하기
print(X_train['계절'].unique())
print(X_train['날씨'].unique())
print(X_train['공휴일'].unique())
print(X_train['근무일'].unique())

[1 2 3 4]
[1 2 3 4]
[0 1]
[0 1]


#### 기초통계량 확인하기

In [15]:
print(X_train.describe().T)

        count       mean        std   min      25%     50%      75%       max
계절    10886.0   2.506614   1.116174  1.00   2.0000   3.000   4.0000    4.0000
공휴일   10886.0   0.028569   0.166599  0.00   0.0000   0.000   0.0000    1.0000
근무일   10886.0   0.680875   0.466159  0.00   0.0000   1.000   1.0000    1.0000
날씨    10886.0   1.418427   0.633839  1.00   1.0000   1.000   2.0000    4.0000
온도    10886.0  20.230860   7.791590  0.82  13.9400  20.500  26.2400   41.0000
체감온도  10886.0  23.655084   8.474601  0.76  16.6650  24.240  31.0600   45.4550
습도    10886.0  61.886460  19.245033  0.00  47.0000  62.000  77.0000  100.0000
풍속    10886.0  12.799395   8.164537  0.00   7.0015  12.998  16.9979   56.9969


#### 독립변수와 종속변수의 관계 확인하기

In [17]:
data = pd.concat([X_train, y_train], axis=1)
data.head()

,datetime,계절,공휴일,근무일,날씨,온도,체감온도,습도,풍속,datetime,count
0,2011-01-01 0:00,1,0,0,1,9.84,14.395,81,0.0,2011-01-01 0:00,16
1,2011-01-01 1:00,1,0,0,1,9.02,13.635,80,0.0,2011-01-01 1:00,40
2,2011-01-01 2:00,1,0,0,1,9.02,13.635,80,0.0,2011-01-01 2:00,32
3,2011-01-01 3:00,1,0,0,1,9.84,14.395,75,0.0,2011-01-01 3:00,13
4,2011-01-01 4:00,1,0,0,1,9.84,14.395,75,0.0,2011-01-01 4:00,1


In [20]:
# '계절' 컬럼에 따른 count(자전거 대여량) 합계 구하기
print(data.groupby('계절')['count'].sum())
print('\n>>>> 가을의 자전거 대여량이 봄의 자전거 대여량에 비해 2배임을 확인')

계절
1    312498
2    588282
3    640662
4    544034
Name: count, dtype: int64

>>>> 가을의 자전거 대여량이 봄의 자전거 대여량에 비해 2배임을 확인


In [22]:
# '공휴일' 컬럼에 따른 count(자전거 대여량) 합계 구하기
# 0 : 공휴일 아님, 1 : 공휴일

print(data.groupby('공휴일')['count'].sum())
print('\n>>>> 공휴일이 아닌 날의 자전거 대여량이 압도적으로 높음 확인')

공휴일
0    2027668
1      57808
Name: count, dtype: int64

>>>> 공휴일이 아닌 날의 자전거 대여량이 압도적으로 높음 확인


In [24]:
# '근무일' 컬럼에 따른 count(자전거 대여량) 합계 구하기
# 0 : 근무일 아님, 1 : 근무일

print(data.groupby('근무일')['count'].sum())
print('\n>>>> 근무일인 날의 대여량이 상대적으로 높음 확인')

근무일
0     654872
1    1430604
Name: count, dtype: int64

>>>> 근무일인 날의 대여량이 상대적으로 높음 확인


In [26]:
# '날씨' 컬럼에 따른 count(자전거 대여량) 합계 구하기

print(data.groupby('날씨')['count'].sum())
print('\n>>>> 날씨가 좋을수록 자전거 대여량이 높아짐 확인')

날씨
1    1476063
2     507160
3     102089
4        164
Name: count, dtype: int64

>>>> 날씨가 좋을수록 자전거 대여량이 높아짐 확인


### 3.2 전처리

#### 파생변수 만들기

- datetime 컬럼 분해 > 요일 데이터 새로 생성

In [32]:
# 날짜 타입으로 변경
X_train['datetime'] = pd.to_datetime(X_train['datetime'])

# X_train 의 'datetime' 컬럼에서 연도(year), 월(month), 일(day), 시간(hour), 요일(dayofweek) 변수 생성
X_train['year']      = X_train['datetime'].dt.year
X_train['month']     = X_train['datetime'].dt.month
X_train['day']       = X_train['datetime'].dt.day
X_train['hour']      = X_train['datetime'].dt.hour
X_train['dayofweek'] = X_train['datetime'].dt.dayofweek

# 각각의 컬럼에서 중복제거된 값 확인
print(X_train['year'].unique())        # 2011, 2012 년도 데이터
print(X_train['month'].unique())       # 1 ~ 2월  데이터
print(X_train['day'].unique())         # 1 ~ 19일 데이터
print(X_train['hour'].unique())        # 0시 ~ 23시 데이터
print(X_train['dayofweek'].unique())   # 0~6

[2011 2012]
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[5 6 0 1 2 3 4]


In [34]:
# 파생변수 포함된 독립변수와 종혹변수 통합한 data2 만들기
data2 = pd.concat([X_train, y_train], axis=1)
data2.head()

,datetime,계절,공휴일,근무일,날씨,온도,체감온도,습도,풍속,year,month,day,hour,dayofweek,datetime,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,2011,1,1,0,5,2011-01-01 0:00,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,1,5,2011-01-01 1:00,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,2011,1,1,2,5,2011-01-01 2:00,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,3,5,2011-01-01 3:00,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,2011,1,1,4,5,2011-01-01 4:00,1


새로 만든 컬럼별 자전거 대여량의 합계 계산 후, 각 독립변수에 따른 count 값의 추세 확인
- year, month, day, hour, dayofweek

In [36]:
print(data2.groupby('year')['count'].sum())
print('\n>>>> 2012년 대여량이 2011년보다 약 2배 많음 확인')

year
2011     781979
2012    1303497
Name: count, dtype: int64

>>>> 2012년 대여량이 2011년보다 약 2배 많음 확인


In [39]:
print(data2.groupby('month')['count'].sum())
print('\n>>>> 1,2월의 경우 적은편에 속하나, 3~12월은 상대적으로 대여량 증가')
print('     대체로 대여량이 비슷한 값을 가지고 있으므로 분석 대상에서 제외')

month
1      79884
2      99113
3     133501
4     167402
5     200147
6     220733
7     214617
8     213516
9     212529
10    207434
11    176440
12    160160
Name: count, dtype: int64

>>>> 1,2월의 경우 적은편에 속하나, 3~12월은 상대적으로 대여량 증가
     대체로 대여량이 비슷한 값을 가지고 있으므로 분석 대상에서 제외


In [42]:
X_train = X_train.drop(columns = ['month'])

In [44]:
print(data2.groupby('day')['count'].sum())
print('\n>>>> 1~19일의 데이터가 모두 비슷한 값 유지하므로 분석 대상에서 제외')

day
1     103692
2     105381
3     111561
4     112335
5     109115
6     108600
7     105486
8     102770
9     108041
10    111645
11    111146
12    109257
13    111448
14    112406
15    115677
16    109837
17    118255
18    108437
19    110387
Name: count, dtype: int64

>>>> 1~19일의 데이터가 모두 비슷한 값 유지하므로 분석 대상에서 제외


In [45]:
X_train = X_train.drop(columns = ['day'])

In [47]:
print(data2.groupby('hour')['count'].sum())
print('\n>>>> 출근 시간대인 8-9시, 퇴근 시간대인 17-18시에 자전거 대여량이 상대적으로 높음 확인')

hour
0      25088
1      15372
2      10259
3       5091
4       2832
5       8935
6      34698
7      96968
8     165060
9     100910
10     79667
11     95857
12    116968
13    117551
14    111010
15    115960
16    144266
17    213757
18    196472
19    143767
20    104204
21     79057
22     60911
23     40816
Name: count, dtype: int64

>>>> 출근 시간대인 8-9시, 퇴근 시간대인 17-18시에 자전거 대여량이 상대적으로 높음 확인


In [49]:
print(data2.groupby('dayofweek')['count'].sum())
print('\n>>>> 요일별 대여량이 큰 차이가 없으므로 분석 대상에서 제외')

dayofweek
0    295296
1    291985
2    292226
3    306401
4    302504
5    311518
6    285546
Name: count, dtype: int64

>>>> 요일별 대여량이 큰 차이가 없으므로 분석 대상에서 제외


In [50]:
X_train = X_train.drop(columns = ['dayofweek'])

In [53]:
# 테스트 데이터에도 동일 적용
X_test['datetime'] = pd.to_datetime(X_test['datetime'])
X_test['year'] = X_test['datetime'].dt.year
X_test['hour'] = X_test['datetime'].dt.hour

#### 불필요한 컬럼 삭제하기

- 기존의 datetime 컬럼 삭제
- X_test 의 'datetime '컬럼 값은 X_test_dt 변수에 별도 저장

In [56]:
X_test_dt = X_test['datetime']

In [57]:
# datetime 컬럼 삭제
X_train = X_train.drop(columns = ['datetime'])
X_test  = X_test.drop(columns = ['datetime'])
y_train = y_train.drop(columns = ['datetime'])

### 3.3 학습 및 평가

#### 데이터 분리하기

In [58]:
from sklearn.model_selection import train_test_split

X_TRAIN, X_VAL, Y_TRAIN, Y_VAL = train_test_split(X_train, y_train, test_size = .2, random_state = 42)

# 분리된 데이터의 행/열 구조 확인
print(X_TRAIN.shape, X_VAL.shape, Y_TRAIN.shape, Y_VAL.shape)

(8708, 10) (2178, 10) (8708, 1) (2178, 1)


#### 데이터 학습 및 하이퍼 파라미터 튜닝

In [59]:
from xgboost import XGBClassifier, XGBRegressor

#### [1] XGB 에서 대표적으로 사용하는 하이퍼 파라미터 작성

- n_estimators = 100, max_depth = 3

In [60]:
model1 = XGBRegressor(n_estimators = 100, max_depth = 3, random_state = 42)
model1.fit(X_TRAIN, Y_TRAIN)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### [2] 하이퍼 파라미터 값 변경 후 학습

In [61]:
model2 = XGBRegressor(n_estimators = 200, max_depth = 5, random_state = 42)
model2.fit(X_TRAIN, Y_TRAIN)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### 결과 예측하기

In [62]:
y_pred1 = pd.DataFrame(model1.predict(X_test), columns = ['count'])
y_pred2 = pd.DataFrame(model2.predict(X_test), columns = ['count'])

예측된 y_pred 변수는 음수로 예측될 가능성이 있으므로 음수는 모두 0으로 변경

In [63]:
y_pred1[y_pred1['count'] < 0] = 0
y_pred2[y_pred2['count'] < 0] = 0

print(pd.DataFrame(y_pred1).head())
print(pd.DataFrame(y_pred2).head())

      count
0  7.836666
1  0.000000
2  0.000000
3  0.000000
4  0.000000
       count
0  15.401005
1   0.000000
2   0.000000
3   0.000000
4   0.000000


검증용 세트 구성

In [66]:
Y_VAL_PRED1 = pd.DataFrame(model1.predict(X_VAL), columns = ['count'])
Y_VAL_PRED2 = pd.DataFrame(model2.predict(X_VAL), columns = ['count'])

# 'count' 컬럼 값이 음수인 데이터 추출하여 0으로 바꾸기
Y_VAL_PRED1[Y_VAL_PRED1['count'] < 0] = 0
Y_VAL_PRED2[Y_VAL_PRED2['count'] < 0] = 0

#### 모델 평가하기

In [67]:
from sklearn.metrics import r2_score

In [68]:
print(r2_score(Y_VAL, Y_VAL_PRED1))
print(r2_score(Y_VAL, Y_VAL_PRED2)) # 정답, 예상값 비교 > 두번째 모델 최종 선정

0.9101562004243906
0.9438512727544658


### 결과 제출하기

In [74]:
final = pd.concat([X_test_dt, y_pred2], axis=1)
final.to_csv('data/1234.csv', index=False)

In [ ]:
pd.read_csv('data/1234.csv', encoding = 'c[]')